# T検定

$$ t　= \frac{\bar{x}-\mu}{\frac{u}{\sqrt{n}}} $$

- \bar{x}: 標本平均
- \mu: 母平均
- u**2: 不偏分散
- n: サンプル数


In [39]:
import numpy as np
import scipy.stats as st

data = np.array([-1, -1, 0, 0, 1, 3, 5, 6, 7, 7])
kitaichi = sum(data)/len(data)
print(f'E(x) = {kitaichi}')

print(f'E(x) = {np.mean(data)}')

hyojunGosa = np.sqrt(np.var(data, ddof=1) / len(data))
print(f'V(x) = {np.sqrt(np.var(data, ddof=1))}')
print(f'標準誤差: {hyojunGosa}')

t_value = (kitaichi - 0) / hyojunGosa
print(f't_value = {t_value}')

# t_検定
# t_value => statistic=2.5861309700971096
# p_value => pvalue=0.02939946072683463
st.ttest_1samp(data, popmean=0)

E(x) = 2.7
E(x) = 2.7
V(x) = 3.3015148038438356
標準誤差: 1.0440306508910548
t_value = 2.5861309700971096


Ttest_1sampResult(statistic=2.5861309700971096, pvalue=0.02939946072683463)

# ANOVA (Analysis Of Variance)

分散分析は、「選択肢を変えることによって、平均値(期待値)が変化するか」を検定したい時に利用する。
